# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_eng = pd.read_csv("data/movie_reviews_eng.csv")
df_eng

,Movie / TV Series,Year,Synopsis,Review
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale..."
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su..."
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio..."
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ..."
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun..."
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina..."
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi..."
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and..."
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ..."


In [3]:
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.rename(columns={'Titre': 'Title', 'Année': 'Year', 'Critiques': 'Review'}, inplace=True)
    df_fr['Original Language'] = 'French' #Add the "Original Language" column 
    df_fr

,Title,Year,Synopsis,Review,Original Language
0,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",French
1,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
2,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",French
3,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French
4,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
5,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",French
6,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",French
7,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French
8,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
9,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French


### Get data from `.csv` files and then preprocess data

In [4]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    
    #English dataset
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng.rename(columns={'Movie / TV Series': 'Title'}, inplace=True)
    df_eng['Original Language'] = 'English' #Add the "Original Language" column 

    #French dataset
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.rename(columns={'Titre': 'Title', 'Année': 'Year', 'Critiques': 'Review'}, inplace=True)
    df_fr['Original Language'] = 'French' #Add the "Original Language" column 
    
    
    #Spanish dataset
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.rename(columns={'Título': 'Title', 'Año': 'Year', 'Sinopsis': 'Synopsis','Críticas': 'Review'}, inplace=True)
    df_sp['Original Language'] = 'Spanish' #Add the "Original Language" column 

    #concatenate dataframes
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)

    
    return df

df = preprocess_data()

In [5]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
22,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Spanish
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French
13,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [6]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en'

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True)


    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [20]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language'] == 'French', 'Review']
fr_reviews_en = translate(str(fr_reviews), fr_en_model,fr_en_tokenizer)


# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language'] == 'French', 'Synopsis']
fr_synopsis_en = translate(str(fr_synopsis), fr_en_model,fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language'] == 'Spanish', 'Review']
es_reviews_en = translate(str(es_reviews), es_en_model, es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language'] == 'Spanish', 'Synopsis']
es_synopsis_en = translate(str(es_synopsis), es_en_model, es_en_tokenizer)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

# Overwrite 'Review' column
df.loc[df['Original Language'] == 'French', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'Spanish', 'Review'] = es_reviews_en

# Overwrite 'Synopsis' column
df.loc[df['Original Language'] == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df['Original Language'] == 'Spanish', 'Synopsis'] = es_synopsis_en

In [7]:
def extract_translate(text, model, tokenizer):
    
    sent = text.tolist()
    listt = []
    for x in sent:
        i = translate(str(x), model, tokenizer)
        listt.append(i)

    return listt


In [8]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language'] == 'French', 'Review']
fr_reviews_en = extract_translate(fr_reviews, fr_en_model,fr_en_tokenizer)


# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language'] == 'French', 'Synopsis']
fr_synopsis_en = extract_translate(fr_synopsis, fr_en_model,fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language'] == 'Spanish', 'Review']
es_reviews_en = extract_translate(es_reviews, es_en_model, es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language'] == 'Spanish', 'Synopsis']
es_synopsis_en = extract_translate(es_synopsis, es_en_model, es_en_tokenizer)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

# Overwrite 'Review' column
df.loc[df['Original Language'] == 'French', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'Spanish', 'Review'] = es_reviews_en

# Overwrite 'Synopsis' column
df.loc[df['Original Language'] == 'French', 'Synopsis'] = fr_synopsis_en
df.loc[df['Original Language'] == 'Spanish', 'Synopsis'] = es_synopsis_en

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [9]:
df.tail()

,Title,Year,Synopsis,Review,Original Language
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [19]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "finiteautomata/bertweet-base-sentiment-analysis" 
sentiment_classifier = pipeline(model= model_name)


# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)[0]

    return "Positive" if result['label'] == 'POS' else "Negative"

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [20]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

df['Sentiment'] = df['Review'].apply(lambda text: analyze_sentiment(text, sentiment_classifier))

In [21]:
counts_column1 = df['Sentiment'].value_counts()
counts_column1

Positive    15
Negative    15
Name: Sentiment, dtype: int64

In [22]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",French,Negative
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,Negative
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,Positive
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,Positive
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,Negative
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,Positive
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English,Positive
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,Positive
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish,Negative
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish,Negative


In [25]:
# export the results to a .csv file
#df.to_csv("result/reviews_with_sentiment.csv", index=False)

In [26]:
# export the results to a .csv file

import os

directory = "result"

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(directory):
    os.makedirs(directory)

# Now save the DataFrame to a CSV file in the 'result' directory
file_path = os.path.join(directory, "reviews_with_sentiment.csv")
df.to_csv(file_path, index=False)
